# Train- Data Loading

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms


## Train and Validation Dataset

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
# /content/gdrive/MyDrive/Colab Notebooks/Dataset/Processed.zip

In [4]:
!unzip -q -o /content/gdrive/MyDrive/Colab\ Notebooks/Dataset/Processed.zip -d datasets


#### Loading npy files

In [5]:
def load_file(path):
  np.load(path).astype(np.float32)

In [7]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.49, 0.248),
    transforms.RandomAffine(degrees=(-5,5), translate=(0, 0.05), scale=(0.9, 1.1)),
    transforms.RandomResizedCrop((224,224), scale=(0.35, 1))
])

In [8]:
val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.49, 0.248)
])

In [10]:
train_dataset = torchvision.datasets.DatasetFolder("/content/datasets/train", loader=load_file, extensions="npy", transform=train_transforms)

In [11]:
val_dataset = torchvision.datasets.DatasetFolder("/content/datasets/val", loader=load_file, extensions="npy", transform=val_transforms)

In [13]:
fig, axis = plt.subplots(2,2, figsize=(10,10))

for row in range(2):
  for column in range(2):
    random_index = np.random.randint(0,24000)
    x_ray, label = train_dataset[random_index]


torchvision.datasets.folder.DatasetFolder